In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
import datetime
import time
from sklearn.neural_network import MLPRegressor

In [24]:
house_df = pd.read_csv('../../data/properties_2016.csv')

D:\Program Files\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
# delete features with too many missing values
nan = house_df.isnull().sum()
nan = nan[nan < 2900000]
nan_feature = nan.index.tolist()
house_nan_df = house_df.loc[:,nan_feature]

In [26]:
# delete categorical features
cate_feature = ['airconditioningtypeid','fips','heatingorsystemtypeid','pooltypeid7','propertycountylandusecode','propertylandusetypeid',
                'propertyzoningdesc','rawcensustractandblock','regionidcity','regionidcounty','regionidneighborhood','regionidzip',
                'censustractandblock']

for i in cate_feature:
    nan_feature.remove(i)

nan_cate_feature = nan_feature

house_nan_cate_df = house_nan_df.loc[:,nan_cate_feature]

In [27]:
# delete highly correlated features

highcorr_feature = ['structuretaxvaluedollarcnt','threequarterbathnbr','finishedfloor1squarefeet','finishedsquarefeet12','finishedsquarefeet15', 'finishedsquarefeet50','taxvaluedollarcnt']

for i in highcorr_feature:
    nan_cate_feature.remove(i)

nan_cate_co_feature = nan_cate_feature

house_nan_co_cate_df = house_nan_cate_df.loc[:,nan_cate_co_feature]

# rename the dataframe, after deleting many features
house_new_df = house_nan_co_cate_df

In [28]:
house_new_df.shape

(2985217, 21)

In [29]:
nan_zero = 'poolcnt'
house_new_df[nan_zero] = house_new_df[nan_zero].fillna(value=0)

In [30]:
nan_mean = ['parcelid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid','calculatedbathnbr', 'calculatedfinishedsquarefeet', 
            'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude','lotsizesquarefeet',
            'roomcnt', 'unitcnt', 'yearbuilt', 'numberofstories','assessmentyear','landtaxvaluedollarcnt', 'taxamount']

house_new_df[nan_mean] = house_new_df[nan_mean].fillna(house_new_df[nan_mean].mean())

In [ ]:
house_new_df.describe()

In [9]:
cols_to_norm = ['bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr','calculatedfinishedsquarefeet', 'fireplacecnt',
                'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude','lotsizesquarefeet', 'poolcnt', 'roomcnt', 
                'unitcnt', 'yearbuilt', 'numberofstories','assessmentyear', 'landtaxvaluedollarcnt', 'taxamount']


for col in cols_to_norm:
    house_new_df[col] = (house_new_df[col] - house_new_df[col].mean())/(house_new_df[col].std())

# house_new_df[cols_to_norm] = house_new_df[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.std())

In [ ]:
house_new_df.describe()

In [10]:
train_df = pd.read_csv('../../data/train_2016_v2.csv')

In [ ]:
train_df

In [11]:
join_df = train_df.merge(house_new_df, how='left', on='parcelid')

In [ ]:
join_df

In [12]:
# training data set: prediction target

target = join_df.loc[:,'logerror']

In [13]:
# training data set: attributes

train = join_df.iloc[:,2:]

# feature = join_df.iloc[:,2:]

In [14]:
# convert datetime into float

for num,str in enumerate(train.transactiondate):
        date_time = datetime.datetime.strptime(str,'%Y-%m-%d')
        str = time.mktime(date_time.timetuple())
        train.set_value(num,'transactiondate',str)
        
train['transactiondate'] = train['transactiondate'].astype(float)

In [15]:
def z_score_norm(df, feature_list):
    """ Dot the z-score method on selected features in feature_list. 
        z = (x - mean)/std
    
    Args:
        df (pandas.DataFrame): data set
        feature_list   (list): a list of features, each element is a string
    """
    
    for col in feature_list:
        df[col] = (df[col] - df[col].mean())/(df[col].std())

In [ ]:
z_score_norm(train, ['transactiondate'])
train['transactiondate']

In [17]:
# Models


'''
model = linear_model.LinearRegression()
model = model.fit(feature, target)
'''


model = MLPRegressor()
model = model.fit(train, target)


"""
model = linear_model.Ridge(alpha=1.0)
model = model.fit(train, target)
"""

'''
model = RandomForestRegressor(max_depth=6, random_state=0)
model = model.fit(feature, target)
'''

'\nmodel = RandomForestRegressor(max_depth=6, random_state=0)\nmodel = model.fit(feature, target)\n'

In [18]:
model

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [19]:
sample = pd.read_csv("../../data/sample_submission.csv")

In [ ]:
sample1 = sample.ParcelId

In [ ]:
sample1.columns

In [ ]:
sample1.columns = ['parcelid']

In [ ]:
sample1 = sample1.to_frame()

In [ ]:
sample1.columns

In [ ]:
sample1.columns

In [20]:
sample1 = sample.ParcelId
sample1 = sample1.to_frame()
sample1.columns = ['parcelid']
sample1.describe()
df_test = sample1.merge(house_new_df, on='parcelid', how='left')

In [21]:
del df_test['parcelid']

In [22]:
def date_transform(df_test):
    for num,str in enumerate(df_test.transactiondate):
        date_time = datetime.datetime.strptime(str,'%Y-%m-%d')
        str = time.mktime(date_time.timetuple())
        df_test.set_value(num,'transactiondate',str)
    df_test['transactiondate'] = df_test['transactiondate'].astype(float) 
    z_score_norm(df_test, ['transactiondate'])
    test_matrix = df_test.as_matrix()
    return test_matrix

In [23]:
result = sample1.copy()
result.columns = ['ParcelId']


df_test1 = df_test.copy()
df_test1.insert(0,'transactiondate','2016-10-15')
df_test1 = date_transform(df_test1)
pred1 = model.predict(df_test1)
pred1 = np.asarray(pred1)
result.insert(1,'201610',pred1)
del df_test1


df_test2 = df_test.copy()
df_test2.insert(0,'transactiondate','2016-11-15')
df_test2 = date_transform(df_test2)
pred2 = model.predict(df_test2)
pred2 = np.asarray(pred2)
result.insert(2,'201611',pred2)
del df_test2


df_test3 = df_test.copy()
df_test3.insert(0,'transactiondate','2016-12-15')
df_test3 = date_transform(df_test3)
pred3 = model.predict(df_test3)
pred3 = np.asarray(pred3)
result.insert(3,'201612',pred3)
del df_test3


df_test4 = df_test.copy()
df_test4.insert(0,'transactiondate','2017-10-15')
df_test4 = date_transform(df_test4)
pred4 = model.predict(df_test4)
pred4 = np.asarray(pred4)
result.insert(4,'201710',pred4)
del df_test4


df_test5 = df_test.copy()
df_test5.insert(0,'transactiondate','2017-11-15')
df_test5 = date_transform(df_test5)
pred5 = model.predict(df_test5)
pred5 = np.asarray(pred5)
result.insert(5,'201711',pred5)
del df_test5


df_test6 = df_test.copy()
df_test6.insert(0,'transactiondate','2017-12-15')
df_test6 = date_transform(df_test6)
pred6 = model.predict(df_test6)
pred6 = np.asarray(pred6)
result.insert(6,'201712',pred6)
del df_test6

# result.columns = ['ParcelId', '201610']

result.columns = ['ParcelId', '201610', '201611', '201612', '201710','201711','201712']
result.to_csv('../predictions/sample12.csv',index=False,header=True)  # need to change filename per run

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').